# Classical DFT

In [4]:
import numpy as np

In [ ]:
def LennardJones(eps, sigma, r):
# Lennard-Jones potential function generator    
    return 4 * eps * ((sigma / r) ** 12 - (sigma / r) ** 6)


def MF_LennardJones(eps, sigma, z):
    # Mean-field contribution to the free energy with lennard-jones potential
    
    beta = 1.0  # Inverse temperature (1/kT), set to 1 for simplicity
    if abs(z) <= sigma:
        return - 6 *np.pi *beta * eps* sigma**2 *(1/5)
    if abs(z) > sigma:
        return - 4 *np.pi *beta * eps* sigma**2 * (1/5 *(sigma/z)**10 - (1/2)*(sigma/z)**4)



def V_ex(z, h, eps_wf, sigma_wf):
    def w(z, eps_wf, sigma_wf):
        return (2 * np.pi * eps_wf) * ((2/45) * (sigma_wf / z)**9 - (1/3)* (sigma_wf / z)**3)

    return w(z, eps_wf, sigma_wf) + w(h - z, eps_wf, sigma_wf)

def chem_potential(nb, eps_w, sigma_w, T):
    k = 1.0  # Boltzmann constant, set to 1 for simplicity
    return k*T * np.ln((1 - nb * sigma_w**3)/nb) + k*T*1 / (1 - nb * sigma_w**3) - (32/9) * np.pi * eps_w * sigma_w**3 * nb
                                   
                            


In [10]:
def denisty_profile(nz, h, z, V_ex, pot_mu, eps, sigma):
    beta = 1.0  # Inverse temperature (1/kT), set to 1 for simplicity
    
    r = np.arange(0.25, h, 0.25)
    integral = np.trapz(nz * MF_LennardJones(eps, sigma, abs(z - r)), r, dx=0.25)
    
    return (1 - nz* sigma**3)* np.exp(- beta * (V_ex(r) - pot_mu)) * np.exp(nz * sigma**3/(1 - nz* sigma**3)) * np.exp(- beta * integral)

In [ ]:
def dOmega(nz, h, z, V_ex, pot_mu, eps, sigma):
    beta = 1.0  # Inverse temperature (1/kT), set to 1 for simplicity
    r = np.arange(0.25, h, 0.25)
    integral = np.trapz(nz * MF_LennardJones(eps, sigma, abs(z - r)), r, dx=0.25)
    return np.ln(nz/(1 - nz* sigma**3)) + nz* sigma**3/(1 - nz* sigma**3) + beta * (V_ex(z) - pot_mu) + beta * integral
    



In [13]:
def convergence(nz_new, nz_old, tol= 1e-6):
    diff = np.abs(nz_new - nz_old)/(np.abs(nz_old))
    return diff < tol